In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in datasets
all_reviews = pd.read_csv('all_reviews_merged.csv')

In [3]:
# Drop the 'train_test' column
all_reviews = all_reviews.drop('train_test', axis=1)

In [4]:
# Add attribute columns
all_reviews['sustainability (y/n)'] = 0

In [5]:
# Preview Data
all_reviews

,reviewId,asin,date_cleaned,username,title,keyword,verified,rating_cleaned,text,sustainability (y/n)
0,R18NC04B57B1WM,B09LRY9C88,2022-04-08,Tina Adcox,awesome!,Movie,True,5.0,great movie,0
1,R1MPI8QQ1R4QZY,B09LRY9C88,2022-04-08,Amazon Customer,boring and ridiculous!,Movie,True,1.0,The plot. Gave up after 20 minutes. Does anyon...,0
2,R31HH2207Z6W8A,B09LRY9C88,2022-04-08,Hydman,Cool,Movie,True,5.0,Greatovie but over hyped,0
3,R36RICDJG8MPW1,B09LRY9C88,2022-04-08,M. Fields,Great movie,Movie,True,5.0,Do buy this film. It really is great but don't...,0
4,R3BBWMNCFP15OM,B09LRY9C88,2022-04-08,Bama Road Rage,I was never a fan of the spider-man franchise,Movie,True,2.0,...and I'm still not a fan after watching this...,0
...,...,...,...,...,...,...,...,...,...,...
1115232,R2R6K97W4CDB3X,B08C2X57ZH,2019-09-16,Claudio,NaN,best Watch,True,5.0,"L'orologio è molto bello, non solo in consider...",0
1115233,R1BNSB3WLR7SGQ,B08C2X57ZH,2019-09-15,marco,NaN,best Watch,True,5.0,"AMAZON professionali come sempre, sono piacevo...",0
1115234,R1R10QW28I1RIE,B08C2X57ZH,2019-09-07,Anna,NaN,best Watch,True,5.0,Very impressed with the quality and look would...,0
1115235,R2FF892TZK2DZO,B08C2X57ZH,2019-08-14,Cliente Amazon,NaN,best Watch,True,4.0,Bello.ma da utilizzare senza impegno,0


In [6]:
all_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115237 entries, 0 to 1115236
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   reviewId              1115237 non-null  object 
 1   asin                  1115237 non-null  object 
 2   date_cleaned          1115237 non-null  object 
 3   username              1115174 non-null  object 
 4   title                 1047991 non-null  object 
 5   keyword               1115237 non-null  object 
 6   verified              1115237 non-null  bool   
 7   rating_cleaned        1115237 non-null  float64
 8   text                  1115064 non-null  object 
 9   sustainability (y/n)  1115237 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 77.6+ MB


In [7]:
##start ESGERT MODEL

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
 
tokenizer_name = "ESGBERT/EnvironmentalBERT-environmental" #path to download from HF
model_name = "ESGBERT/EnvironmentalBERT-environmental"
# the tokenizer prepares the text for the model and the model classifies the text-
 
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, max_len=512)
# The pipeline combines tokenizer and model to one process. 
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer) # set device=0 to use GPU
 
# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
print(pipe("Scope 1 emissions are reported here on a like-for-like basis against the 2013 baseline and exclude emissions from additional vehicles used during repairs.", padding=True, truncation=True))

/home/jupyter-svanhemert00/.local/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


[{'label': 'environmental', 'score': 0.9979760050773621}]


In [10]:
#get sample data to test pipeline 
reviews = all_reviews.iloc[:,8]
reviews_list = reviews.tolist()
print(reviews)
#made text into a list, wasn't working as a dataframe
print(reviews_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# You can input single sentences or arrays of sentences into the pipeline.
reviews_list
reviews_pipe = pipe(reviews_list)
print(reviews_pipe)

In [ ]:
# Classify subset of sentences.
# The padding and trunctuation parameters help us with classifying texts of different length.
sub_sentences = reviews_list[:100] # takes around 20 seconds
# full sentences take around 5mins WITHOUT GPU (see next tutorial for speed ups)
env = pipe(sub_sentences, padding=True, truncation=True)

# You might only want the labels.
env_labels = [x["label"] for x in env]

In [ ]:
import pandas as pd

# Let's look at the results. We use a dataframe for this purpose.
data_env = pd.DataFrame({"sentence": sub_sentences, "environmental": env_labels})
# Which sentences are labeled as environmental?
data_env[data_env["environmental"] == "environmental"]

In [ ]:
print(data_env.groupby("environmental").count())
data_env.groupby("environmental").count().plot(kind="bar")